# 演習0 準備

《学修項目》
*   ソーシャルメディアデータ，検索エンジンとSNS，オープンデータ

<a name="ref"></a>
《参考文献，参考書籍》
*   [1] TBA


## 演習0-1 Wikipediaの情報を取得する例

*     [Python Wrapper for Wikipedia](https://pypi.org/project/Wikipedia-API/)

### (step 1) Python環境チェック， pipライブラリインストール，path調整（オプション）

In [ ]:
#実行環境の確認
!cat /etc/issue
!free -h
!cat /proc/cpuinfo | grep "model name"

In [ ]:
#Python環境の確認
import platform
print("python " + platform.python_version())

In [ ]:
# Pythonライブラリpathをチェックしておく
import sys
import pprint

pprint.pprint(sys.path)

In [ ]:
# 最初に実行するときは，以下の pip コマンドによって必要なライブラリをインストールする
!pip install wikipedia
!pip install wikipedia-api

In [ ]:
# インストールされた場所をチェック（Location:）
!pip show wikipedia
!pip show wikipedia-api

In [ ]:
# 【ローカル環境のAnaconda上のjupyter notebookなどの場合】
# 上のLocation: の結果に応じて，ライブラリpathを追加する（以下はhogeユーザのpyenv環境でライブラリpathを追加する例）
sys.path.append('/Users/hoge/.pyenv/versions/3.7.12/lib/python3.7/site-packages')
pprint.pprint(sys.path)

### (step 2) インターネット上のオープンデータ（Wikipedia）にアクセスして情報取得

In [ ]:
# wikipediaライブラリをインポート
import wikipedia

# 使用言語を日本語にして，「ビッグデータ」のページを検索
wikipedia.set_lang("ja")
pprint.pprint(wikipedia.search("ビッグデータ"))

# ページタイトルと，サマリ部分を取得
wp = wikipedia.page("ビッグデータ")
print(wp.title)
print(wp.summary)

In [ ]:
# wikipedia-apiライブラリをインポート
import wikipediaapi

# 使用言語を英語にして，「Big_data」のページを検索
wiki_wiki = wikipediaapi.Wikipedia('en')
page_py = wiki_wiki.page('Big_data')

# セクションレベルに応じて * を付ける関数の定義
def print_sections(sections, level=0):
        for s in sections:
                print("%s: %s - %s" % ("*" * (level + 1), s.title, s.text[0:40]))
                print_sections(s.sections, level + 1)

# 当該ページのセクション情報をすべて取得して表示する
print("All Sections")
print_sections(page_py.sections)
print("\n")

# ページが属するカテゴリをすべて取得する関数の定義
def print_categories(page):
        categories = page.categories
        for title in sorted(categories.keys()):
            print("%s: %s" % (title, categories[title]))

# 当該ページが属するカテゴリをすべて取得して表示する
print("Categories")
print_categories(page_py)

## 演習0-2 CSVオープンデータをダウンロードした後，統計的処理を行う例

*     [新型コロナダッシュボード](https://www.stopcovid19.jp/)

### (step 0a) 【準備：Google colab.の場合】 Googleドライブへのファイルアクセス許可について

In [ ]:
# GoogleDriveへのファイルアクセスを許可するには，左側の　[実行ボタン] をクリックし， [Googleドライブに接続]>[アカウント選択]>[接続]の順でクリックしてください．
from google.colab import drive
drive.mount('/content/drive')

# Googleアカウントへの認証が成功すると，以下に Mounted at /content/drive と表示されます．

In [ ]:
# Google colab.からアクセス可能な，　GoogleDriveの現在のフォルダの状況（/content/drive/MyDrive）は以下を実行することで表示されます．
!ls -al /content/drive/MyDrive

### (step 0b) 【準備：Windowsローカル環境上でのjupyter notebookの場合】 GNU版 wgetコマンドをインストールする

以下の攻略記事などを参考にして，GNU版 wget をインストールし，コマンドPATHを通す
*   [Windows10 wgetをインストールして使用する手順](https://mebee.info/2020/05/25/post-10477/)
*   [Windowsでもwgetを使いたい](https://jinyuki.hateblo.jp/entry/2020/12/19/202202)

### (step 1) Data準備フォルダを作成し，オープンデータをダウンロードする

In [ ]:
# 【Google colab.の場合】 マイドライブの直下に， Data/ex1-2 という名前のフォルダを作成する（既に存在しているならばスキップ）
!mkdir /content/drive/MyDrive/Data
!mkdir /content/drive/MyDrive/Data/ex1-2

# CSVデータを Data/ex1-2 フォルダへダウンロードする
!wget -c -P '/content/drive/MyDrive/Data/ex1-2' https://www.stopcovid19.jp/data/covid19japan_beds/latest.csv

# CSVファイルがダウンロードできたことを確認する
!ls -al /content/drive/MyDrive/Data/ex1-2

In [ ]:
# 【ローカル環境のjupyter notebookの場合】 カレントディレクトリの直下に， Data/ex1-2 という名前のフォルダを作成する（既に存在しているならばスキップ）
!mkdir ./Data
!mkdir ./Data/ex1-2

# CSVデータを Data/ex1-2 フォルダへダウンロードする
# DATA: 療養状況等及び入院患者受入病床数等に関する調査について｜厚生労働省

## wgetコマンドがインストールされている場合
!wget -c -P './Data/ex1-2' https://www.stopcovid19.jp/data/covid19japan_beds/latest.csv

# CSVファイルがダウンロードできたことを確認する
!ls -al ./Data/ex1-2

In [ ]:
## wgetコマンドが使えない場合，以下の download_file 関数を使う
#import os
#import pprint
#import time
import urllib.error
import urllib.request

def download_file(url, dst_path):
    try:
        with urllib.request.urlopen(url) as web_file:
            data = web_file.read()
            with open(dst_path, mode='wb') as local_file:
                local_file.write(data)
    except urllib.error.URLError as e:
        print(e)

url = 'https://www.stopcovid19.jp/data/covid19japan_beds/latest.csv'
dst_path = './Data/ex1-2/latest.csv'
download_file(url, dst_path)

# CSVファイルがダウンロードできたことを確認する
!ls -al ./Data/ex1-2

total 24
drwxr-xr-x 2 root root  4096 Jul 31 05:20 .
drwxr-xr-x 3 root root  4096 Jul 31 05:20 ..
-rw-r--r-- 1 root root 14953 Jul 31 07:11 latest.csv


In [ ]:
# Pandasライブラリをインポート
import pandas as pd

# CSVデータをPandas data frameで読み出し
df = pd.read_csv('/content/drive/MyDrive/Data/ex1-2/latest.csv')  #注意：ダウンロードした場所によってpathは適宜変更する

##df.head(47)  # 47都道府県すべてを表示する場合
df.head(10)  # 先頭10行だけ表示する場合

In [ ]:
# PCR検査陽性者数 が 最大の 都道府県名 と 数値 を表示する
max_idx = df['PCR検査陽性者数'].idxmax()
print(df['都道府県名'].iloc[max_idx])
print("PCR検査陽性者数", df['PCR検査陽性者数'].iloc[max_idx], "人")

In [ ]:
# PCR検査陽性者あたり入院者　を求めて，新たな列 '入院レート' を作成する（注：既に 入院率 列があるが，整数値となっている）
df['入院レート'] = (df['入院者数'] / df['PCR検査陽性者数']) * 100

# 入院レート　　が 最大の 都道府県名 と 数値 を表示する
max_rate_idx = df['入院レート'].idxmax()
print(df['都道府県名'].iloc[max_rate_idx])
print("入院レート", '{:.2f}'.format(df['入院レート'].iloc[max_rate_idx]), "％") #小数点2位まで

In [ ]:
# 全ての列の「要約統計量」を表示する
df.describe()

# memo